# Getting started with the Qiskit-Braket provider

### Qiskit to Braket mapping

![qiskit-to-braket-diagram](./data/qiskit-braket-mapping.png)

We first start by importing all required classes and functions for this notebook. We also start the cost tracker to print the costs at the end of this notebook.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.random import random_circuit
from qiskit.visualization import plot_histogram

from braket.tracking import Tracker
from qiskit_braket_provider import BraketLocalBackend, BraketProvider, to_braket

# Use Braket SDK Cost Tracking to estimate the cost to run this example
t = Tracker().start()

### Access Braket devices from Qiskit 

`BraketProvider` class gives you a method `backends` to access backends that are available through Braket SDK.

Let's get available devices to use by Qiskit

In [ ]:
provider = BraketProvider()
print(provider.backends())

If you want to explore what is available by specific contraints, you can specify query arguments to `backends` method of provider.
Arguments are fully compatible with Braket's `get_device` method. See the documentation at [braket.aws.aws_device.AwsDevice.get_devices](https://amazon-braket-sdk-python.readthedocs.io/en/stable/_apidoc/braket.aws.aws_device.html#braket.aws.aws_device.AwsDevice.get_devices). For example, you can retrieve the list of online simulators via:

In [ ]:
online_simulators_backends = provider.backends(statuses=["ONLINE"], types=["SIMULATOR"])
online_simulators_backends

For prototyping it is usually a good practice to use simulators 
to set up workflow of your program and then change it to real device.
We can access local simulator by creating instance of class `BraketLocalBackend`

In [ ]:
local_simulator = BraketLocalBackend()
local_simulator

Any backend can be instantiated via the `get_backend` method of the provider. Here is an example where we create a Backend object for the IonQ `Aria 1` device.

In [ ]:
Aria_1 = provider.get_backend("Aria 1")

### Running circuits on Braket devices

Let's first create Qiskit circuit. We will start with a Bell circuit.

In [ ]:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.draw()

This circuit can be used to submit a task to the local simulator. In the [tutorials](https://github.com/qiskit-community/qiskit-braket-provider/tree/main/docs/tutorials) associated to the Qiskit-Braket provider, we will use the Braket taxonomy:
- a task is an atomic request to a device or a simulator. 
- an hybrid job is a mean to run hybrid quantum-classical algorithms requiring both classical AWS resources and quantum processing units (QPUs). See [What is a Hybrid Job](https://docs.aws.amazon.com/braket/latest/developerguide/braket-what-is-hybrid-job.html) for more details.

Here, quantum tasks are analogous to Qiskit jobs, which is why tasks have a `job_id` property. If a task has been submitted to Braket managed device, `job_id` will return a Task ARN (Amazon Resource number) which identifies the task and allows to retrieve it in the Braket Console and in your notebooks. 

In [ ]:
task = local_simulator.run(qc, shots=10)

Results are returned via a `Result` object, from which you can extract counts and plot them in a histogram.

In [ ]:
results = task.result()
plot_histogram(results.get_counts())

Each single shot measurements are also retrievable.

In [ ]:
results.get_memory()

More complex circuits on devices can be submitted to the Braket devices and simulators. Behind the scenes, the qiskit transpiler will adapt the circuits such that they are executable on all devices. In the following, we will focus on running circuit in the IonQ Aria-1 device. 

We will start with generating random circuit and printing it out.

In [ ]:
qiskit_random_circuit = random_circuit(4, 5, seed=42)
qiskit_random_circuit.draw(fold=-1)

Each device has a set of supported operations which are accessible via the backend.

In [ ]:
aria_supported_gates = Aria_1.get_gateset()
print(aria_supported_gates)

In [ ]:
braket_random_circuit = to_braket(qiskit_random_circuit, basis_gates=aria_supported_gates)
print(braket_random_circuit)

We printed the Braket Circuit object that will be used to create the quantum task on the Aria-1 device. To submit the task, you do not have to create a Braket circuit, this is done internal when you submit the Qiskit circuit to the Backend.

In [ ]:
aria_task = Aria_1.run(qiskit_random_circuit, shots=10)

If you do not want to wait for the task completion and return to it later, you can use `retrieve_job` method on device to get job object. For this, you will need to know the job_id (which is the task ARN that identifies the task in the AWS cloud).

In [ ]:
task_arn = aria_task.job_id()
retrieved_task = Aria_1.retrieve_job(task_id=task_arn)

From the task, you can check its status: 

In [ ]:
retrieved_task.status()

and get the results as done previously: 

In [ ]:
random_circuit_results = retrieved_task.result()
plot_histogram(random_circuit_results.get_counts())

In [ ]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print(
    "Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing "
    "unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated "
    "charges do not factor in any discounts or credits, and you may experience additional charges "
    "based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2)."
)
print(
    f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.3f} USD"
)